In [1]:
import pandas as pd
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import utils

%load_ext autoreload
%autoreload 2


In [2]:
df_sch, meta_sch = pyreadstat.read_sav('data/PISA2022_SCH_QQQ.SAV')

In [3]:
import os

with open('data/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [9]:
   
cols = utils.meta2docs(meta_sch)

cols_vectorstore = Chroma.from_documents(cols, OpenAIEmbeddings(),  persist_directory="./chroma_db_cols_vectorstore4")

cols_retriever = cols_vectorstore.as_retriever()

In [10]:
test_data = [
    {'question':'Which country has the highest average  teacher student ratio? How much is it? ','expected_columns': ['STRATIO', 'CNT'], 'expected_answer': 28.95},
    {'question':'Which country has the highest average class sizes? How much is it?','expected_columns': ['CLSIZE', 'CNT'], 'expected_answer': 42.65 },    
    {'question':'In how many languages ​​was the questionnaire filled out?', 'expected_columns': ['LANGTEST_QQQ'], 'expected_answer': 54},
    {'question':'What is Hungary average teacher-student ratio?', 'expected_columns': ['STRATIO', 'CNT'], 'expected_answer': 10.83},
    {'question':'What is OECD average teacher-student ratio?', 'expected_columns': ['STRATIO', 'OECD'], 'expected_answer':  12.551608},
    {'question':'Where is average teacher-student ratio in Hungary compared to the OECD average in terms of average teacher-student ratio? Give the difference as number.','expected_columns': ['STRATIO', 'OECD', 'CNT'], 'expected_answer': -1.721608 },    
    #{'question':'how many education systems were involved in the survey? ','expected_columns': ['STRATIO', 'CNT'], 'expected_answer': 28.95},
    {'question':'how many countries were involved in the survey?','expected_columns': ['CNT'], 'expected_answer': 80},
    {'question':'how many countries that are OECD memeber were involved in the survey? ','expected_columns': ['OECD', 'CNT'], 'expected_answer': 37},
    {'question':'how many schools were involved in the survey?','expected_columns': ['CNTSCHID'], 'expected_answer': 21629},
    {'question':'what is the relation between school autonomy  and teacher participation? Please give linear regression coefficient as result','expected_columns': ['SCHAUTO','TCHPART'], 'expected_answer':  0.2216 },
]


In [11]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)

eval_res = utils.execute_tests(llm, test_data, df_sch, meta_sch, cols_retriever)

- Country
- Teacher student ratio
hint - Country
['REGION', 'OECD', 'SCHLTYPE', 'CNT']
hint - Teacher student ratio
['STRATIO', 'SMRATIO', 'PROPMATH', 'PROMGMT']
content="def run(data):\n    data = data.dropna(subset=['CNT', 'STRATIO'])\n    highest_avg_ratio = data.groupby('CNT')['STRATIO'].mean().max()\n    return float(highest_avg_ratio)" response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 275, 'total_tokens': 319}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-e74dbd7d-3d43-47d0-9901-04f27b16ad6f-0' usage_metadata={'input_tokens': 275, 'output_tokens': 44, 'total_tokens': 319}
def run(data):
    data = data.dropna(subset=['CNT', 'STRATIO'])
    highest_avg_ratio = data.groupby('CNT')['STRATIO'].mean().max()
    return float(highest_avg_ratio)
- Country
- Average class size
hint - Country
['REGION', 'OECD', 'SCHLTYPE', 'CNT']
hint - Average class size
['MCLSIZE', 'CLSIZE', 'SCHSIZE', 'STRATIO']


KeyError: ['CLSIZ']

In [41]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)

eval_res = utils.execute_tests(llm, [test_data[-1]], df_sch, meta_sch, cols_retriever)

1. School autonomy (independent variable)
2. Teacher participation (dependent variable)

Linear regression coefficient: This will be the coefficient calculated by running a linear regression analysis on the data collected for school autonomy and teacher participation.
['School autonomy (WLE)', "Extent structures your school's math programme: Discretion of individual teachers", 'How often you/others in [school management team]: Taking actions to support co-operation among teachers to develop new teaching practices', 'School\xa0Preparation for Remote Instruction, In Response to Pandemic\xa0(WLE)', 'Proportion of mathematics teachers at school', 'Student-teacher ratio']
content="```python\nimport pandas as pd\nimport numpy as np\nfrom sklearn.linear_model import LinearRegression\n\ndef run(data):\n    X = data['SCHAUTO'].values.reshape(-1, 1)\n    y = data['SC201Q04JA'].values\n\n    model = LinearRegression()\n    model.fit(X, y)\n\n    return float(model.coef_[0])\n\n# Example usage:\n#

In [42]:
eval_res

[{'question': 'what is the relation between school autonomy  and teacher participation? Please give linear regression coefficient as result',
  'expected_columns': ['SCHAUTO', 'TCHPART'],
  'expected_answer': 0.2216,
  'found_cols': ['SCHAUTO'],
  'found_cols_ratio': 0.5,
  'pipeline_result': -0.009560998571684062,
  'hint_cols': ['SCHAUTO',
   'SC188Q09JA',
   'SC201Q04JA',
   'SCPREPAP',
   'PROPMATH',
   'STRATIO'],
  'error': -0.23116099857168404}]

In [44]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)

eval_res = utils.execute_tests(llm, [test_data[-1]], df_sch, meta_sch, cols_retriever)

1. School ID
2. School autonomy level
3. Teacher participation level

Linear regression coefficient: 0.65
['Intl. School ID', 'School type', 'School autonomy (WLE)', "Extent structures your school's math programme: Discretion of individual teachers", 'How often you/others in [school management team]: Taking actions to ensure that teachers take responsibility for improving their teaching skills', 'School\xa0Preparation for Remote Instruction, In Response to Pandemic\xa0(WLE)', 'Student-mathematics teacher ratio', 'Student-teacher ratio']
content="def run(df):\n    from sklearn.linear_model import LinearRegression\n    \n    X = df['SCHAUTO'].values.reshape(-1, 1)\n    y = df['SC201Q05JA'].values\n    \n    model = LinearRegression()\n    model.fit(X, y)\n    \n    return float(model.coef_[0])" response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 334, 'total_tokens': 400}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprob

In [45]:
eval_res

[{'question': 'what is the relation between school autonomy  and teacher participation? Please give linear regression coefficient as result',
  'expected_columns': ['SCHAUTO', 'TCHPART'],
  'expected_answer': 0.2216,
  'found_cols': ['SCHAUTO'],
  'found_cols_ratio': 0.5,
  'pipeline_result': 0.01855577452108215,
  'hint_cols': ['CNTSCHID',
   'SCHLTYPE',
   'SCHAUTO',
   'SC188Q09JA',
   'SC201Q05JA',
   'SCPREPAP',
   'SMRATIO',
   'STRATIO'],
  'error': -0.20304422547891784}]